#Import Libraries

In [2]:
%tensorflow_version 2.x

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pathlib

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.layers import MaxPool2D, MaxPool1D
from tensorflow.keras.layers import Dropout, BatchNormalization

from google.colab import drive

print("Module Loaded.")
print("TensorFlow Version :{}".format(tf.__version__))
print("NumPy Version :{}".format(np.__version__))
print("Matplotlib Version :{}".format(plt.matplotlib.__version__))

TensorFlow 2.x selected.
Module Loaded.
TensorFlow Version :2.0.0
NumPy Version :1.17.4
Matplotlib Version :3.1.2


In [3]:
drive.mount('/content/drive')

#data_path = pathlib.Path('/content/drive/Shared drives/scsa_2019_b/Project_Ai-KEA/data/furniture')\
#!cp -r /content/drive/Shared\ drives/scsa_2019_b/Project_Ai-KEA/data_resize .

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [13]:
data_path = pathlib.Path('/content/furniture')

class_name = np.array([item.name for item in data_path.glob('*')])
print(class_name)

['standing_lamp' 'entertainment_center' 'table' 'makeup_table' 'curtain'
 'bed' 'bookshelves' 'sofa' 'chair' 'rug']


In [5]:
import os

cwd = os.getcwd()
print(cwd)

!unzip /content/drive/Shared\ drives/scsa_2019_b/Project_Ai-KEA/data_resize/furniture.zip

/content
Archive:  /content/drive/Shared drives/scsa_2019_b/Project_Ai-KEA/data_resize/furniture.zip
  inflating: furniture/table/table407.jpg  
  inflating: furniture/table/table404.jpg  
  inflating: furniture/table/table409.jpg  
  inflating: furniture/table/table386.jpg  
  inflating: furniture/table/table397.jpg  
  inflating: furniture/table/table399.jpg  
  inflating: furniture/table/table385.jpg  
  inflating: furniture/table/table408.jpg  
  inflating: furniture/table/table400.jpg  
  inflating: furniture/table/table394.jpg  
  inflating: furniture/table/table387.jpg  
  inflating: furniture/table/table398.jpg  
  inflating: furniture/table/table392.jpg  
  inflating: furniture/table/table406.jpg  
  inflating: furniture/table/table405.jpg  
  inflating: furniture/table/table388.jpg  
  inflating: furniture/table/table391.jpg  
  inflating: furniture/table/table402.jpg  
  inflating: furniture/table/table389.jpg  
  inflating: furniture/table/table390.jpg  
  inflating: furnit

# Resize Image

#ImageGenerator

In [0]:
img_size = 112
batch_n = 200


In [0]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                                width_shift_range=0.3,
                                height_shift_range=0.3,
                                #horizontal_flip=True, 
                                #vertical_flip=True,
                                #rotation_range=30,
                                #zoom_range=0.1,
                                #brightness_range=[0.8,1.2],
                                validation_split=0.2,
                                rescale=1./255)

In [8]:
train_data_generate = image_generator.flow_from_directory(
                        directory=str(data_path),
                        batch_size=batch_n,
                        shuffle=True,
                        target_size=(img_size, img_size),
                        classes = list(class_name),
                        subset='training')
test_data_generate = image_generator.flow_from_directory(
                        directory=str(data_path),
                        batch_size=batch_n,
                        shuffle=True,
                        target_size=(img_size, img_size),
                        classes = list(class_name),
                        subset='validation')



Found 3705 images belonging to 10 classes.
Found 921 images belonging to 10 classes.


#Load VGG

import model

##fine tuning

freeze layers

make ai-kea model

In [0]:

def model_cnn_batchnormal():
  model = tf.keras.models.Sequential()
  # conv 0
  model.add(tf.keras.layers.Conv2D(64,3,padding='same',activation='relu',input_shape=(112,112,3))) 
  model.add(tf.keras.layers.Conv2D(64,3,padding='same',activation='relu')) 
  model.add(tf.keras.layers.BatchNormalization(momentum=0.85))             # BN
  model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
  # conv 2
  model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
  model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
  model.add(tf.keras.layers.BatchNormalization(momentum=0.85))             # BN
  model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
  # conv 3
  model.add(tf.keras.layers.Conv2D(512, 3, padding='same', activation='relu')) 
  model.add(tf.keras.layers.BatchNormalization(momentum=0.85))             # BN
  model.add(tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)))
  # conv 4
  model.add(tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu')) 
  model.add(tf.keras.layers.BatchNormalization(momentum=0.85))             # BN
  model.add(tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)))
 
  model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu')) 
  model.add(tf.keras.layers.BatchNormalization(momentum=0.85))             # BN
  model.add(tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)))
  # dense layers
  model.add(tf.keras.layers.Flatten(name='flatten'))
  model.add(tf.keras.layers.Dense(len(class_name), activation='softmax', name='dense_10'))

  return model

ai_kea = model_cnn_batchnormal()


#model compile & train

In [0]:
epoch_n = 100

In [0]:
from tensorflow.keras.optimizers import SGD

In [0]:
def lr_schedule_dbase(epoch):
    lr = 1e-2
    if epoch/epoch_n >  0.8:
        lr *= 1e-4
    elif epoch/epoch_n > 0.6:
        lr *= 1e-3
    elif epoch/epoch_n > 0.4:
        lr *= 1e-2
    elif epoch/epoch_n > 0.2:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [0]:


from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
import numpy as np

F_LR_CNTRL = True  # learning rate controled optimizer
#F_LR_CNTRL = False  # default optimizer

if F_LR_CNTRL:
  # optimizser의 'learning_rate='를 정의한 함수로 지정
  opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule_dbase(0))
  #opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule_dbase(0))
  # learning schedule callback
  lrate = LearningRateScheduler(lr_schedule_dbase)
  callbacks_list = [lrate]
  #lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0,
  #                              patience=5, min_lr=0.5e-6)
  #callbacks_list = [lrate, lr_reducer]
else:
  opt = 'adam'
  callbacks_list = []


In [0]:
ai_kea.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
history = ai_kea.fit_generator(
    train_data_generate,
    epochs = epoch_n,
    validation_data = test_data_generate
)

Epoch 1/100
17/19 [=========================>....] - ETA: 2s - loss: 2.5300 - accuracy: 0.2442

In [0]:
print(np.max(history.history['val_accuracy']))